In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

from io import StringIO
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
#import warnings

# Basic data cleaning
Only keep headlines and category for now Remove special characters from headlines and convert to lowercase

In [2]:
dataset_path = Path.cwd() / Path('dataset/News_Category_Dataset_v2.json')
df = pd.read_json(dataset_path, lines = True)

reduced_df = df.drop(df[df['headline'].str.len() == 0].index, axis=0)

reduced_df = reduced_df[['headline', 'category']]

cleaned_df = pd.DataFrame(reduced_df)

def clean_headline(original_headline):
    # remove special characters
    cleaned_headline = re.sub(r'\W', ' ', original_headline)
    cleaned_headline = re.sub(r'\s+', ' ', original_headline)

    cleaned_headline = cleaned_headline.lower()
    return cleaned_headline

cleaned_df['headline'] = reduced_df['headline'].apply(clean_headline)

cleaned_df.head()

,headline,category
0,there were 2 mass shootings in texas last week...,CRIME
1,will smith joins diplo and nicky jam for the 2...,ENTERTAINMENT
2,hugh grant marries for the first time at age 57,ENTERTAINMENT
3,jim carrey blasts 'castrato' adam schiff and d...,ENTERTAINMENT
4,julianna margulies uses donald trump poop bags...,ENTERTAINMENT


In [4]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y = encoder.fit_transform(cleaned_df['category'])
print(y[:5])

[ 6 10 10 10 10]


In [5]:
categories = cleaned_df['category']
titles = cleaned_df['headline']
N = len(titles)
print('Number of news',N)

Number of news 200847


In [6]:
labels = list(set(categories))
print('possible categories',labels)

possible categories ['SPORTS', 'FOOD & DRINK', 'ENTERTAINMENT', 'GOOD NEWS', 'SCIENCE', 'RELIGION', 'EDUCATION', 'CULTURE & ARTS', 'WELLNESS', 'PARENTING', 'WEDDINGS', 'MONEY', 'ENVIRONMENT', 'FIFTY', 'WORLD NEWS', 'ARTS & CULTURE', 'CRIME', 'WOMEN', 'HOME & LIVING', 'PARENTS', 'STYLE & BEAUTY', 'LATINO VOICES', 'IMPACT', 'COLLEGE', 'STYLE', 'BLACK VOICES', 'MEDIA', 'COMEDY', 'HEALTHY LIVING', 'ARTS', 'WEIRD NEWS', 'DIVORCE', 'QUEER VOICES', 'TRAVEL', 'TECH', 'BUSINESS', 'TASTE', 'GREEN', 'POLITICS', 'THE WORLDPOST', 'WORLDPOST']


In [13]:
from sklearn.model_selection import train_test_split
sample = cleaned_df

x_train, x_test, y_train, y_test = train_test_split(
    np.array(sample['headline']), 
    np.array(sample['category_id']), 
    test_size=0.05, # seperate 5 % test
    random_state=42
)

x_train, x_val, y_train, y_val = train_test_split(
    np.array(x_train), 
    np.array(y_train), 
    test_size=3/19, # this evens out to 80% train 15% validation
    random_state=42
)

print('train:', len(x_train))
print('val:', len(x_val))
print('test:', len(x_test))

train: 160677
val: 30127
test: 10043


In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [21]:
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

In [22]:
text_clf = text_clf.fit(x_train, y_train)

In [23]:
print('Predicting...')
predicted = text_clf.predict(x_val)

Predicting...


In [24]:
from sklearn import metrics

print('accuracy_score',metrics.accuracy_score(y_val,predicted))
print('Reporting...')

accuracy_score 0.39512729445348027
Reporting...
